In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras import optimizers
# 転移学習したモデルの中で一番精度が高かったものを読み込む。
###TODO_転移学習して最も精度が高かったモデルのファイル名をinput_modelに設定してください。
input_model = "sample.h5"
model = models.load_model("/content/drive/My Drive/Colab Notebooks/transfer_learning/"+input_model)
model.summary()

In [0]:
#データの準備
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

categories = ["プリン", "カービィ"]
nb_classes = len(categories)

###TODO_numpy_converterで出力したファイル名をinput_numpyに設定###
input_numpy = "sample.npy"
X_train, X_test, Y_train, Y_test = np.load("/content/drive/My Drive/Colab Notebooks/"+input_numpy, allow_pickle=True)

#データの正規化
X_train = X_train.astype("float") / 255
X_test = X_test.astype("float") / 255

#kerasで扱えるようにcategoriesをベクトルに変換
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

#conv_baseのblock5のみtrainableをTrueにする。
conv_base = model.layers[0]
model.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == "block5_conv1":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

#モデルのコンパイル
model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.RMSprop(lr=1e-5),
             metrics=["acc"])
model.summary()

# ModelCheckpointを設定する
modelCheckpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Colab Notebooks/model/finetuning/model_{epoch:02d}.h5', verbose=1, period=10)

#ModelCheckpointで出力されるモデルの保存先フォルダを作成する
try:
  os.makedirs("/content/drive/My Drive/Colab Notebooks/model/finetuning/")
except FileExistsError:
  pass

#学習を実行
model = model.fit(X_train, Y_train, epochs=100, batch_size=64, validation_data=(X_test, Y_test), callbacks=[modelCheckpoint])


In [0]:
#学習結果をグラフで表示する。
import matplotlib.pyplot as plt

acc = model.history['acc']
val_acc = model.history['val_acc']
loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

# 新しいセクション